In [1]:
# Import our data, and separate our dependent variable
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import patsy as pt
data_path = r".\DATA\pakistanClean.csv"
data = pd.read_csv(data_path)

In [2]:
data.head()

,Unnamed: 0,eventid,iyear,imonth,iday,region,region_txt,provstate,city,latitude,...,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,Date,Month,TTP
0,1,200712030005,2007,12,3,6,South Asia,Balochistan,Qilla Abdullah district,30.803630,...,NaN,CETIS,-9,-9,0,-9,NaN,2007-12-03,Dec,NaN
1,2,200712040005,2007,12,4,6,South Asia,Khyber Pakhtunkhwa,Peshawar,34.006004,...,NaN,CETIS,-9,-9,0,-9,NaN,2007-12-04,Dec,NaN
2,3,200712060008,2007,12,6,6,South Asia,Balochistan,Dera Bugti,29.034412,...,NaN,CETIS,0,0,0,0,NaN,2007-12-06,Dec,0.0
3,4,200712080003,2007,12,8,6,South Asia,Balochistan,Khuzdar,27.809921,...,NaN,CETIS,-9,-9,0,-9,NaN,2007-12-08,Dec,NaN
4,5,200712090002,2007,12,9,6,South Asia,Balochistan,Nasirabad,28.458421,...,NaN,CETIS,-9,-9,0,-9,NaN,2007-12-09,Dec,NaN


In [3]:
work_data = data[(data['TTP']==0) | (data['TTP'] ==1)]
work_data['dMonth']= work_data.Month.astype('category').cat.codes
work_data.head(5)

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
<ipython-input-3-6b73c5ff7af5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_data['dMonth']= work_data.Month.astype('category').cat.codes


,Unnamed: 0,eventid,iyear,imonth,iday,region,region_txt,provstate,city,latitude,...,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,Date,Month,TTP,dMonth
2,3,200712060008,2007,12,6,6,South Asia,Balochistan,Dera Bugti,29.034412,...,CETIS,0,0,0,0,NaN,2007-12-06,Dec,0.0,2
22,23,200712230005,2007,12,23,6,South Asia,Khyber Pakhtunkhwa,Mingora,34.774479,...,CETIS,0,0,0,0,NaN,2007-12-23,Dec,1.0,2
23,24,200712270001,2007,12,27,6,South Asia,Punjab,Rawalpindi,33.594013,...,CETIS,0,1,0,1,NaN,2007-12-27,Dec,0.0,2
24,25,200801010014,2008,1,1,6,South Asia,Federally Administered Tribal Areas,South Waziristan district,32.320237,...,CETIS,0,0,0,0,NaN,2008-01-01,Jan,1.0,4
25,26,200801050006,2008,1,5,6,South Asia,Federally Administered Tribal Areas,Jandola,32.333344,...,CETIS,0,0,0,0,NaN,2008-01-05,Jan,1.0,4


In [4]:
test_data = data[pd.isna(data['TTP'])]

In [5]:
# Create regression arrays
Y, X = pt.dmatrices("TTP ~ C(iyear) + C(provstate) + multiple + success + suicide + attacktype1 + C(targtype1) + weaptype1 + weapsubtype1 + nkill + nkillus + nkillter + nwound + nwoundus + nwoundte + C(Month)", data = work_data)
#C(targsubtype1)
#Began with - ("TTP ~ C(iyear) + C(imonth) + C(provstate) + multiple + success + suicide + attacktype1 + C(targtype1) + C(targsubtype1) + weaptype1 + weapsubtype1 + nkill + nkillus + nkillter + nwound + nwoundus + nwoundte + Month", data = work_data)

    #removed targsubtype - increased accuracy from .799 to .809

In [6]:
# Randomly create train and test data
x, xt, y, yt = train_test_split(X, Y, test_size = 0.25,random_state=3)

In [7]:
# Import libraries, including the SVC classifier
from sklearn.svm import SVC #support vector classifier

y = np.ravel(y) # Reduce y to a 1-D array

# Generate a model with a C value of 10.
# C is our error penalty term.
model = SVC(C=10, kernel='linear') # Linear, rbf, polynomial, and sigmoid kernels
#kernel is how we dea with no linearities

reg = model.fit(x, y) # Fit the mo del
pred = reg.predict(xt) # Make predictions with test data
print("\nModel accuracy is %s\n" %
      str(round(accuracy_score(pred, np.ravel(yt)), 3)))


Model accuracy is 0.809



In [8]:
#model could not handle NAN values. Changed all values to 0 - representing not TTP attacks.
test_data['TTP'] = 0

<ipython-input-8-fc7fb0416aab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['TTP'] = 0


In [9]:
pred_Y, pred_X = pt.dmatrices("TTP ~ C(iyear) + C(provstate) + multiple + success + suicide + attacktype1 + C(targtype1) + weaptype1 + weapsubtype1 + nkill + nkillus + nkillter + nwound + nwoundus + nwoundte + C(Month)", data = test_data)
#pred_X = test_data['TTP']

In [10]:
test_pred = reg.predict(pred_X)
test_pred

array([0., 1., 0., ..., 0., 1., 0.])

In [11]:
#pct TTP attacls
TTP_attacks = test_pred.sum()/len(test_pred)
TTP_attacks

0.5342545285647933